# Project: Answering Business Questions using SQL

## Creating Helper Functions

First we import sqlite and pandas

In [2]:
import sqlite3
import pandas as pd

making a function to take SQL query as an argument and returns a pandas dataframe of that query

In [3]:
def run_query(q):
    with sqlite3.connect('chinook.db') as conn:
        return pd.read_sql(q, conn)

making a function that takes a SQL command as an argumant and executs it using sqlite module

In [4]:
def run_command(c):
    with sqlite3.connect('chinook.db') as conn:
        conn.isolation_level = None
        conn.execute(c)

creating a function that calls run_query() function to return a list of all tables and views in the databasee

In [5]:
def show_tables():
    q = '''
    SELECT
        name,
        type
    FROM sqlite_master
    WHERE type IN ("table","view");
    '''
    return run_query(q)

showing the tables

In [6]:
show_tables()

,name,type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


## Selecting Albums to Purchase

query to return each genre, with the number of tracks sold in the USA in absolute numbers and in percentages

In [7]:
genres_num_track = '''
WITH track_sold_us AS(
    SELECT il.track_id AS track_id, SUM(il.quantity) AS tracks_sold
    FROM invoice_line AS il
    INNER JOIN invoice AS i ON i.invoice_id = il.invoice_id
    WHERE i.billing_country = 'USA'
    GROUP BY 1
    ORDER BY 2 DESC
    ),
    track_genre AS(
    SELECT t.track_id AS track_id, g.name AS genre_name
    FROM track AS t 
    INNER JOIN genre AS g ON g.genre_id = t.genre_id
    )

SELECT tg.genre_name AS genre_name, 
       SUM(tsu.tracks_sold) AS tracks_sold,
       ROUND(CAST(SUM(tsu.tracks_sold) AS FLOAT) / (SELECT SUM(tracks_sold) FROM track_sold_us) * 100, 2) AS percentage_sold
FROM track_genre AS tg 
INNER JOIN track_sold_us AS tsu ON tsu.track_id = tg.track_id
GROUP BY 1
ORDER BY 2 DESC;
'''
run_query(genres_num_track)

,genre_name,tracks_sold,percentage_sold
0,Rock,561,53.38
1,Alternative & Punk,130,12.37
2,Metal,124,11.80
3,R&B/Soul,53,5.04
4,Blues,36,3.43
5,Alternative,35,3.33
6,Pop,22,2.09
7,Latin,22,2.09
8,Hip Hop/Rap,20,1.90
9,Jazz,14,1.33


Out of the albums proposed, we can choose based on the most selling genre tracks. The album that can be profitable is Red Tone, which is punk, second most selling genre. This is followed by Slim Jim Bites, whose genre is Blues, fifth most sellling genre. As for the third album that can be included, based on the data, we must choose Pop, which sits on the seventh position

## Analyzing Employee Sales Performance

Finding the total dollar amount of sales assigned to each sales support agent within the company

In [8]:
sales_agent = '''
WITH sales_employee_list AS(
    SELECT * FROM employee WHERE title = "Sales Support Agent"
    ),
    sales_rep_sales AS(
    SELECT c.support_rep_id AS sales_id, SUM(i.total) AS total_sales
    FROM customer AS c 
    INNER JOIN invoice AS i ON i.customer_id = c.customer_id
    GROUP BY 1
    )

SELECT sel.first_name || " " || sel.last_name AS sales_employee,
       sel.hire_date,
       srs.total_sales
FROM sales_employee_list AS sel 
INNER JOIN sales_rep_sales AS srs ON srs.sales_id = sel.employee_id
ORDER BY 3 DESC
'''
run_query(sales_agent)

,sales_employee,hire_date,total_sales
0,Jane Peacock,2017-04-01 00:00:00,1731.51
1,Margaret Park,2017-05-03 00:00:00,1584.00
2,Steve Johnson,2017-10-17 00:00:00,1393.92


the total sales number can be understood by the data of hire date of the employee. The oldest hire has the most number of sales.

## Analyzing Sales by Country

analyzing the sales data for cutomers from each different country. These include the total number of customers, total value of sales, average value of sales per customer, and the average order value

In [14]:
sales_by_country = '''
WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line AS il
     INNER JOIN invoice AS i ON i.invoice_id = il.invoice_id
     INNER JOIN customer AS c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) AS customers,
        SUM(unit_price) AS total_sales,
        SUM(unit_price) / count(distinct invoice_id) AS average_order,
        SUM(unit_price) / count(distinct customer_id) AS customer_lifetime_value,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );
'''

run_query(sales_by_country)

,country,customers,total_sales,average_order,customer_lifetime_value
0,USA,13,1040.49,7.942672,80.037692
1,Canada,8,535.59,7.047237,66.948750
2,Brazil,5,427.68,7.011148,85.536000
3,France,5,389.07,7.781400,77.814000
4,Germany,4,334.62,8.161463,83.655000
5,Czech Republic,2,273.24,9.108000,136.620000
6,United Kingdom,3,245.52,8.768571,81.840000
7,Portugal,2,185.13,6.383793,92.565000
8,India,2,183.15,8.721429,91.575000
9,Other,15,1094.94,7.448571,72.996000


From the data, we can see an intesting fact on Czechia, which despite low sales, customers tend to buy more. Czechia have potential for growth

## Albums vs Individual Tracks

query to categorize each invoice as either an album purchase or not to calculate the number of invoices and the percentage of invoices

In [17]:
album_tracks = '''
WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;
'''
run_query(album_tracks)

,album_purchase,number_of_invoices,percent
0,no,500,0.814332
1,yes,114,0.185668


about a fifth of the sales is through album purchase